# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: [Dies Natalis Solis Invicti](https://en.wikipedia.org/wiki/Sol_Invictus)
* Reviews: [Befana](https://en.wikipedia.org/wiki/Befana)

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [33]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice
from copy import deepcopy
from tqdm.auto import tqdm
import numpy as np

In [71]:
State = namedtuple('State', ['x', 'o'])
value_dictionary_x = defaultdict(float)
value_dictionary_o = defaultdict(float)
Wins = 0
Loses = 0
Draws = 0

In [35]:
MAGIC = [2, 7, 6, 
         9, 5, 1, 
         4, 3, 8]

In [67]:
def print_board(pos):
    """Nicely prints the board"""
    for r in range(3):
        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print('X ', end='')
            elif MAGIC[i] in pos.o:
                print('O ', end='')
            else:
                print('. ', end='')
        print()
    print()

In [37]:
def win(elements):
    """Checks is elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))

# AI X player

In [38]:
def state_value_x(pos: State):
    """Evaluate state: +1 first player wins"""
    if win(pos.x):
        return 1
    elif win(pos.o):
        return -1
    else:
        return 0

# Make move function for X player 

In [39]:
def make_move_x(state, available):
    
    max_val = float('-inf')
    move = None
    bool = 1

    for tmp in available:

        next_state_x = state.x.union({tmp})
        hashable_next_state = (frozenset(next_state_x), frozenset(state.o))

        if hashable_next_state in value_dictionary_x:
            if value_dictionary_x[hashable_next_state] > max_val:
                max_val = value_dictionary_x[hashable_next_state]
                move = tmp
                bool = 0
    
    if bool: 
        move = choice(list(available))

    return move

# Training phase for X player 

In [40]:
def training_game_x():
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    while available:

        """x turn"""
        x = choice(list(available))
        state.x.add(x)
        trajectory.append(deepcopy(state))
        available.remove(x)

        if win(state.x) or not available:
            break
    
        """o turn"""
        o = choice(list(available))
        state.o.add(o)
        trajectory.append(deepcopy(state))
        available.remove(o)

        if win(state.o):
            break
        
    return trajectory

In [41]:
"""First training"""

epsilon = 0.001

for steps in tqdm(range(1_000_000)):
    trajectory = training_game_x()
    final_reward = state_value_x(trajectory[-1])
    for state in trajectory:
        hashable_state = (frozenset(state.x), frozenset(state.o))
        value_dictionary_x[hashable_state] = value_dictionary_x[hashable_state] + epsilon * (final_reward - value_dictionary_x[hashable_state])

  0%|          | 2043/1000000 [00:00<02:32, 6530.62it/s]

100%|██████████| 1000000/1000000 [02:30<00:00, 6641.11it/s]


# Game tests for X player

In [42]:
def game_x():
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    while available:

        """x turn"""
        x = make_move_x(state, available)
        state.x.add(x)
        trajectory.append(deepcopy(state))
        available.remove(x)

        if win(state.x):
            global Wins 
            Wins += 1
            break
        else:
            if not available: 
                global Draws
                Draws +=1
                break


        """o turn"""
        o = choice(list(available))
        state.o.add(o)
        trajectory.append(deepcopy(state))
        available.remove(o)

        if win(state.o):
            global Loses 
            Loses += 1
            break
        
    return trajectory

In [43]:
for steps in tqdm(range(10_000)):
    trajectory = game_x()

print(f"Wins: {Wins},\nLoses: {Loses},\nDraws: {Draws}")

global Loses 
Loses = 0

global Draws
Draws = 0

global Wins 
Wins = 0

100%|██████████| 10000/10000 [00:01<00:00, 8479.50it/s]

Wins: 9896,
Loses: 0,
Draws: 104


# AI O player

In [59]:
def state_value_o(pos: State):
    
    if win(pos.o):
        return 3
    elif win(pos.x):
        return -1
    else:
        return 2

# Make a move function for a O player

In [60]:
def make_move_o(state, available):
    
    max_val = float('-inf')
    move = None
    bool = 1

    for tmp in available:

        next_state_o = state.o.union({tmp})
        hashable_next_state = (frozenset(state.x), frozenset(next_state_o))

        if hashable_next_state in value_dictionary_o:
            if value_dictionary_o[hashable_next_state] > max_val:
                max_val = value_dictionary_o[hashable_next_state]
                move = tmp
                bool = 0
    
    if bool: 
        move = choice(list(available))

    return move

# Training for O player 

In [61]:
def training_game_o():
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    while available:

        """x turn"""
        x = choice(list(available))
        state.x.add(x)
        trajectory.append(deepcopy(state))
        available.remove(x)

        if win(state.x) or not available:
            break
    
        """o turn"""
        o = choice(list(available))
        state.o.add(o)
        trajectory.append(deepcopy(state))
        available.remove(o)

        if win(state.o):
            break
        
    return trajectory

In [62]:
"""First training"""

epsilon = 0.001

for steps in tqdm(range(1_000_000)):
    trajectory = training_game_o()
    final_reward = state_value_o(trajectory[-1])
    for state in trajectory:
        hashable_state = (frozenset(state.x), frozenset(state.o))
        value_dictionary_o[hashable_state] = value_dictionary_o[hashable_state] + epsilon * (final_reward - value_dictionary_o[hashable_state])

100%|██████████| 1000000/1000000 [02:28<00:00, 6750.94it/s]


# Game tests for O player

In [63]:
def game_o():
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    while available:

        """x turn"""
        x = choice(list(available))
        state.x.add(x)
        trajectory.append(deepcopy(state))
        available.remove(x)

        if win(state.x):
            global Loses 
            Loses += 1
            break
        else:
            if not available: 
                global Draws
                Draws +=1
                break


        """o turn"""
        o = make_move_o(state, available)
        state.o.add(o)
        trajectory.append(deepcopy(state))
        available.remove(o)

        if win(state.o):
            global Wins 
            Wins += 1
            break
        
    return trajectory

In [64]:
for steps in tqdm(range(10_000)):
    trajectory = game_o()

print(f"Wins: {Wins},\nLoses: {Loses},\nDraws: {Draws}")

global Loses 
Loses = 0

global Draws
Draws = 0

global Wins 
Wins = 0

100%|██████████| 10000/10000 [00:01<00:00, 7679.80it/s]

Wins: 8924,
Loses: 167,
Draws: 909


# Test AI X player vs AI O player

In [50]:
def game_AI():
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    while available:

        """x turn"""
        x =  make_move_x(state, available)
        state.x.add(x)
        trajectory.append(deepcopy(state))
        available.remove(x)

        if win(state.x):
            global Loses 
            Loses += 1
            break
        else:
            if not available: 
                global Draws
                Draws +=1
                break


        """o turn"""
        o = make_move_o(state, available)
        state.o.add(o)
        trajectory.append(deepcopy(state))
        available.remove(o)

        if win(state.o):
            global Wins 
            Wins += 1
            break
        
    return trajectory

In [65]:
for steps in tqdm(range(10_000)):
    trajectory = game_AI()

print(f"AI O player wins: {Wins},\nAI X player wins: {Loses},\nDraws: {Draws}")

global Loses 
Loses = 0

global Draws
Draws = 0

global Wins 
Wins = 0

100%|██████████| 10000/10000 [00:01<00:00, 5361.34it/s]

AI O player wins: 0,
AI X player wins: 0,
Draws: 10000


# Defining an interactive game function

In [52]:
def interactive_game(player):
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    while available:

        """x turn"""
        if player == "X":
            bool = 1
            while bool:
                x = int(input("Do your move"))
                if x in available:
                    bool = 0
                else:
                    print("Move not allowed")
        else: 
            x = make_move_x(state, available)

        state.x.add(x)
        trajectory.append(deepcopy(state))
        available.remove(x)
        print_board(state)

        if win(state.x):
            print("X player won")
            break
        else:
            if not available: 
                print("It's a draw")
                break

        """o turn"""
        if player == "O":
            bool = 1
            while bool:
                o = int(input("Do your move"))
                if o in available:
                    bool = 0
                else:
                    print("Move not allowed")
        else: 
            o = make_move_o(state, available)
    
        state.o.add(o)
        trajectory.append(deepcopy(state))
        available.remove(o)
        print_board(state)

        if win(state.o):
            print("O player won")
            break
        
    return trajectory

In [53]:
def interactive_interface():
    
    print("Do you wanna play against AI? Y/N")
    input_user_1 = input()
    
    while input_user_1 == "Y":
        print("Do you wanna play using X or O? X/O")
        input_user_2 = input()
        print("Nice, let's start the game!")
        trajectory = interactive_game(input_user_2)
        print("Do you wanna play against AI? Y/N")
        input_user_1 = input()
    
    print("Goodbye!")


In [ ]:
interactive_interface()